   # Исследование Интернет-мазазина компьютерных игр "Стримчик".

    В нашем распоряжении данные о продажах со всего мира интернет-магазина "Стримчик" до 2016 года. Из открытых источников доступны исторические данные о продажах игр, оценки пользователей и экспертов, жанры и платформы. Все данные хранятся в файле  /datasets/games.csv. Необходимо исследовать данные и выявить закономерности для того, чтобы в 2017 году сделать ставку на потенциально популярный продукт и спланировать рекламные кампании. 
    
    Ход работы:
    
    1) Подготовка данных (обрабатываем дубликаты, пропуски, меняем типы данных, где необходимо)
    2) Добавляем столбец суммарные продажи во всех регионах.
    3) Выбираем нужный актуальный период и провидим исследовательский анализ данных на нем.
    4) Составляем портрет пользователя каждого региона.
    5) Провяем гипотезы.
    6) Пишем итоговые выводы.
   

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from scipy import stats as st
import pylab

#### Шаг 1. Загрузка данных

In [2]:
data = pd.read_csv('/datasets/games.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/games.csv'

In [ ]:
data.shape

In [ ]:
data.head(10)

In [ ]:
data.describe().T

In [ ]:
data.info()

In [ ]:
data.hist(bins=50, figsize=(20, 10));

Записали данные файла в data. Выводим информацию. В данных есть пропущенные значения.

#### Шаг 2. Предобработка данных

In [ ]:
data[data.duplicated()]

Полных дубликатов нет.

In [ ]:
data.isna().sum()

In [ ]:
data['Year_of_Release'].unique()

In [ ]:
data = data.dropna(subset=['Year_of_Release'])

In [ ]:
data['Year_of_Release'].unique()

In [ ]:
data['Year_of_Release'] = data['Year_of_Release'].astype('int')

In [ ]:
ax = data['Year_of_Release'].hist(bins=100)
ax.set(title='data', xlabel='Year_of_Release', ylabel='Sales');

В столбце год выпуска есть пропущенные значения. Скорее всего они не заполнились из-за технической ошибки. Так как в дальнейшем мы будем делить данные по годам, пропуски лучше удалить, их не так много.

In [ ]:
data['Name'] = data['Name'].str.lower()

In [ ]:
data['Platform'] = data['Platform'].str.lower()

In [ ]:
data['Genre'] = data['Genre'].str.lower()

Приводим строковые переменные к нижнему регистру, чтобы было удобнее работать.

In [ ]:
data = data.dropna(subset=['Name', 'Genre'])

Удаляем пропуски в имени и жанре, тк их мало.

In [ ]:
data.isna().sum()

Остались пропущенные значения в трех столбцах, оставим их без изменений, тк их довольно таки большое количество и заполнение их какими-либо данные может повлиять на результат. Скорее всего пропуски получились из-за того, что данные оценки и возрастной рейтинг не были выставлены по какой-то причине.

In [ ]:
data['User_Score'].unique()

In [ ]:
data.shape

In [ ]:
data[data['User_Score'] == 'tbd'].count()

In [ ]:
data.loc[data['User_Score'] == 'tbd', ['User_Score']] = 'nan'

In [ ]:
data['User_Score'] = data['User_Score'].astype('float')

В столбце с оценками пользователей видим элемент tbd, что значит оценка уточняется. Таких элементов слишком много, чтобы их удалить. Поэтому заменим их на nan. После этого сменим тип данных на float.

In [ ]:
data['Rating'].unique()

In [ ]:
data.info()

Таким образом, мы избавились от части пропусков, поменяли типы данных.

In [ ]:
data.shape

In [ ]:
data['Sum_sales'] = data[['EU_sales', 'NA_sales', 'JP_sales', 'Other_sales']].sum(axis=1)

Добавили столбец с суммарными продажами по всем регионам.

#### Шаг 3. Исследовательский анализ данных

In [ ]:
ax = data['Year_of_Release'].hist(bins=100)
ax.set(title='data', xlabel='Year_of_Release', ylabel='sales');

Большинство данных были получены с 2001 до 2016 года. Пик - 2009-2010 года.

In [ ]:
top_pl = data.groupby('Platform')['Sum_sales'].sum().sort_values(ascending=False).head(10).reset_index()
ax = top_pl.plot.bar(x='Platform', y='Sum_sales')
ax.set(title='top_pl', xlabel='Platform', ylabel='sales');

In [ ]:
for pl in top_pl['Platform']:
    plt.figure()
    ax = data.loc[data['Platform'] == pl]['Year_of_Release'].hist(bins=100)
    ax.set(title=pl, xlabel='Year_of_Release', ylabel='sales')
plt.show()

Наиболее популярные платформы ps2, x360, ps3, wii, ds. На их примере видно, что "жизнь" одной платформы примерно 10 лет. 

In [ ]:
data_actual = data[data['Year_of_Release'] > 2013]
data_actual.shape

За актуальный период возьмем 2014-2016 года. Так как сфера видеоигр развивается довольно таки быстро.

In [ ]:
ax = data_actual['Platform'].hist(bins=100, figsize=(20, 10))
ax.set(title='data_actual', xlabel='Platform', ylabel='sales');

In [ ]:
for pl in data_actual['Platform'].unique():
    plt.figure()
    ax = data_actual.loc[data_actual['Platform'] == pl]['Year_of_Release'].hist(bins=6)
    ax.set(title=pl, xlabel='Year_of_Release', ylabel='sales');
plt.show()

В выбранном периоде лидируют ps4 и psv. Продажи ps4, xone, pc, psv растут, значит являются самыми перспективными. Wiiu, ps3, x360, wii падают.

In [ ]:
for pl in ['ps4', 'xone', 'ps3', 'psv', '3ds']:
    a[pl] = pd.DataFrame(data_actual.loc[data_actual['Platform'] == pl]['Sum_sales']).reset_index()['Sum_sales']
a.boxplot(column=['ps4', 'xone', 'ps3', 'psv', '3ds'], figsize=(20, 10));

На боксплоте видно, что самые высокие продажи у ps4 и xone. У них размах до 1.7, выше много выбросов.

In [ ]:
ax = data_actual.loc[data_actual['Platform'] == 'xone'].plot.scatter (x = 'Critic_Score', y = 'Sum_sales', figsize=(10, 5))
ax.set(title='ps3_actual');
data_actual['Critic_Score'].corr(data_actual['Sum_sales'])

In [ ]:
ax = data_actual.loc[data_actual['Platform'] == 'xone'].plot.scatter (x = 'User_Score', y = 'Sum_sales', figsize=(10, 5))
ax.set(title='ps3_actual');
data_actual['User_Score'].corr(data_actual['Sum_sales'])

In [ ]:
ax = data_actual.loc[data_actual['Platform'] == 'ps4'].plot.scatter (x = 'Critic_Score', y = 'Sum_sales', figsize=(10, 5))
ax.set(title='ps4_actual');
data_actual['Critic_Score'].corr(data_actual['Sum_sales'])

In [ ]:
ax = data_actual.loc[data_actual['Platform'] == 'ps4'].plot.scatter (x = 'User_Score', y = 'Sum_sales', figsize=(10, 5))
ax.set(title='ps4_actual');
data_actual['User_Score'].corr(data_actual['Sum_sales'])

Корреляция между отзывами критиков и продажами (0.3) выше,чем между отзывами пользователей и продажами (небольшая отрицательная корелляция). Можно сделать вывод, что на других платформах продажи ниже из-за низкий оценок критиков.

In [ ]:
genre = data_actual.groupby(['Genre'])['Sum_sales'].median()
ax = genre.plot.bar(x = 'Genre', y = genre)
ax.set(title='data_actual', xlabel='Genre', ylabel='sales');

На гистограмме по жанрам видно, что лидирует shooter с большим отрывом. Самые низкие продажи у adventure и puzzle.

#### Шаг 4. Портрет пользователя каждого региона

In [ ]:
data_NA_sales = data_actual.groupby('Platform')['NA_sales'].sum().sort_values(ascending=False).head(5)
x = data_NA_sales[0]
y = data_NA_sales[1]
ax = data_NA_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='NA_sales', xlabel='Platform', ylabel='sales');

In [ ]:
data_EU_sales = data_actual.groupby('Platform')['EU_sales'].sum().sort_values(ascending=False).head(5)
x = data_EU_sales[0]
y = data_EU_sales[1]
ax = data_EU_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='EU_sales', xlabel='Platform', ylabel='sales');

In [ ]:
data_JP_sales = data_actual.groupby('Platform')['JP_sales'].sum().sort_values(ascending=False).head(5)
x = data_JP_sales[0]
y = data_JP_sales[1]
ax = data_JP_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='JP_sales', xlabel='Platform', ylabel='sales');

В Северной Америке лидирует ps4 и x360, в Европе ps4 лидирует с большим отрывом. В Японии же лидирует 3ds.

In [ ]:
data_NA_sales = data_actual.groupby('Genre')['NA_sales'].sum().sort_values(ascending=False).head(5)
x = data_NA_sales[0]
y = data_NA_sales[1]
ax = data_NA_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='NA_sales', xlabel='Genre', ylabel='sales');

In [ ]:
data_EU_sales = data_actual.groupby('Genre')['EU_sales'].sum().sort_values(ascending=False).head(5)
x = data_EU_sales[0]
y = data_EU_sales[1]
ax = data_EU_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='EU_sales', xlabel='Genre', ylabel='sales');

In [ ]:
data_JP_sales = data_actual.groupby('Genre')['JP_sales'].sum().sort_values(ascending=False).head(5)
x = data_JP_sales[0]
y = data_JP_sales[1]
ax = data_JP_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='JP_sales', xlabel='Genre', ylabel='sales');

Первые два места по жанрам в Северной Америке и Европе совпадают: shooter и action. В японии на первом месте role-playing, а на втором action.

In [ ]:
data_NA_sales = data_actual.groupby('Rating')['NA_sales'].sum().sort_values(ascending=False).head(5)
x = data_NA_sales[0]
y = data_NA_sales[1]
ax = data_NA_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='NA_sales', xlabel='Rating', ylabel='sales');

In [ ]:
data_EU_sales = data_actual.groupby('Rating')['EU_sales'].sum().sort_values(ascending=False).head(5)
x = data_EU_sales[0]
y = data_EU_sales[1]
ax = data_EU_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='EU_sales', xlabel='Rating', ylabel='sales');

In [ ]:
data_JP_sales = data_actual.groupby('Rating')['JP_sales'].sum().sort_values(ascending=False).head(5)
x = data_JP_sales[0]
y = data_JP_sales[1]
ax = data_JP_sales.plot.bar(x = 'x', y = 'y')
ax.set(title='JP_sales', xlabel='Rating', ylabel='sales');

В Северной Америке и Европе влияние рейтинга на продажи схоже. В Японии большие продажи игр с рейтингом Т.

В целом видно, что продажи в Северной Америке и Европе зависят от схожих факторов, а Япония довольно сильно отличается. Это связано с культурными особенностями стран.

#### Шаг 5. Проверка гипотез.

In [ ]:
data_actual.loc[data_actual['Platform'] == 'xone']['User_Score']

In [ ]:
data_actual.loc[data_actual['Platform'] == 'pc', 'User_Score']

In [ ]:
alpha = 0.05
results = st.ttest_ind(data_actual.loc[data_actual['Platform'] == 'xone', 'User_Score'].dropna(), \
                       data_actual.loc[data_actual['Platform'] == 'pc', 'User_Score'].dropna())

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу')

Нулевая гиепотеза - средние пользовательские рейтинги платформ Xbox One и PC одинаковые. Альтернативная - они различаются. Применим двухсторонний ttest_ind, чтобы проверить это. По результатам теста отклонить нулевую гипотезу не получилось. Значит можно предположить, что рейтинги одинаковые.

In [ ]:
alpha = 0.05
results = st.ttest_ind(data_actual.loc[data_actual['Genre'] == 'action']['User_Score'].dropna(), \
                       data_actual.loc[data_actual['Genre'] == 'sports']['User_Score'].dropna())

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print('Отвергаем нулевую гипотезу')
else:
    print('Не получилось отвергнуть нулевую гипотезу') 

Нулевая гипотеза - средние пользовательские рейтинги жанров Action и Sports одинаковые.  Альтернативная - они различаются. Применим двухсторонний ttest_ind, чтобы проверить это. По результатам теста отвергаем нулевую гипотезу. Можно предположить, что рейтинги различны.

#### Шаг 5. Общий вывод.

    Предобработка данных:
    1) Полных дубликатов нет.
    2) В столбце год выпуска есть пропущенные значения. Скорее всего они не заполнились из-за технической ошибки. Так как в дальнейшем мы будем делить данные по годам, пропуски лучше удалить, их не так много.
    3) Приводим строковые переменные к нижнему регистру, чтобы было удобнее работать.
    4) Удаляем пропуски в имени и жанре, тк их мало.
    5) Остались пропущенные значения в трех столбцах, оставим их без изменений, тк их довольно таки большое количество и заполнение их какими-либо данные может повлиять на результат. Скорее всего пропуски получились из-за того, что данные оценки и возрастной рейтинг не были выставлены по какой-то причине.
    6) В столбце с оценками пользователей видим элемент tbd, что значит оценка уточняется. Таких элементов слишком много, чтобы их удалить. Поэтому заменим их на nan. После этого сменим тип данных на float.
    
    Исследовательский анализ данных:
    1) Большинство данных были получены с 2001 до 2016 года. Пик - 2009-2010 года.
    2) Наиболее популярные платформы ps2, ds, ps3, wii, x360. На их примере видно, что "жизнь" одной платформы примерно 10 лет.
    3) За актуальный период возьмем 2014-2016 года. Так как сфера видеоигр развивается довольно таки быстро.
    4) В выбранном периоде лидируют ps4 и psv. Продажи ps4, xone, pc, psv растут, значит являются самыми перспективными. Wiiu, ps3, x360, wii падают.
    5) На боксплоте видно, что самые высокие продажи у ps4 и xone. У них размах до 1.7, выше много выбросов.
    6) Корреляция между отзывами критиков и продажами (0.3) выше,чем между отзывами пользователей и продажами (низкая отрицательная корелляция). Можно сделать вывод, что на других платформах продажи ниже из-за низкий оценок критиков.
    7) На гистограмме по жанрам видно, что лидирует shooter с большим отрывом. Самые низкие продажи у adventure и puzzle.
    8) В Северной Америке лидирует ps4 и x360, в Европе ps4 лидирует с большим отрывом. В Японии же лидирует 3ds.
    9) Первые два места по жанрам в Северной Америке и Европе совпадают: shooter и action. В японии на первом месте role-playing, а на втором action.
    10) В Северной Америке и Европе влияние рейтинга на продажи схоже. В Японии большие продажи игр с рейтингом Т.
    11) В целом видно, что продажи в Северной Америке и Европе зависят от схожих факторов, а Япония довольно сильно отличается. Это связано с культурными особенностями стран.
    12) По результатам статистического теста можно предположить, что средние рейтинги платформ Xbox One и PC одинаковые. 
    13) По результатам статистического теста можно предположить, что средние пользовательские рейтинги жанров Action и Sports различны. 